In [3]:
import sys
import torch
from pathlib import Path

print('Python version:', sys.version)
sys.path.append('..')

from src.datasets import *

Python version: 3.10.5 (main, Dec 17 2022, 19:41:24) [Clang 14.0.0 (clang-1400.0.29.202)]


In [4]:
dumb_dataset = DumbImageCaptionDataset(dataset_path=Path('/tmp'), split='train', train_size=20)
dumb_dataset

2023-12-04 18:13:20.983 | DEBUG    | src.datasets.base:__init__:76 - Dataset ImageCaption path: /tmp | Number of samples: 20


In [5]:
dumb_dataset[0][0].shape, dumb_dataset[0][1]

(torch.Size([3, 64, 64]), 'Lorem ipsum dolor')

In [6]:
dataloader = torch.utils.data.DataLoader(dumb_dataset, batch_size=4, shuffle=True)
len(dataloader)

5

In [7]:
for images, captions in dataloader:
    print(images.shape, len(captions))

torch.Size([4, 3, 64, 64]) 4
torch.Size([4, 3, 64, 64]) 4
torch.Size([4, 3, 64, 64]) 4
torch.Size([4, 3, 64, 64]) 4
torch.Size([4, 3, 64, 64]) 4


In [8]:
dataset = get_dataset(
    {
        "name": 'dumb_image_caption',
        "dataset_path": Path('/tmp'),
        "split": 'train',
        "train_size": 20
    }
)
dataset

2023-12-04 18:13:22.717 | DEBUG    | src.datasets.base:__init__:76 - Dataset ImageCaption path: /tmp | Number of samples: 20


In [9]:
dumb_dataset[0][0].shape, dumb_dataset[0][1]

(torch.Size([3, 64, 64]), 'Lorem ipsum dolor sit amet, consect')

In [10]:
dataloader = torch.utils.data.DataLoader(dumb_dataset, batch_size=4, shuffle=True)
len(dataloader)

5

In [11]:
for images, captions in dataloader:
    print(images.shape, len(captions))

torch.Size([4, 3, 64, 64]) 4
torch.Size([4, 3, 64, 64]) 4
torch.Size([4, 3, 64, 64]) 4
torch.Size([4, 3, 64, 64]) 4
torch.Size([4, 3, 64, 64]) 4


## With transforms

In [14]:
import transformers
from torchvision import transforms
from functools import partial


In [15]:
MAX_LEN = 128
vision_transform = transforms.Compose([
    transforms.Resize(244), transforms.CenterCrop(224), transforms.Normalize([0.5], [0.5])
])
caption_transform = partial(transformers.AutoTokenizer.from_pretrained("roberta-base"), return_tensors="pt", padding='max_length', truncation=True, max_length=MAX_LEN)

In [16]:
dumb_dataset = DumbImageCaptionDataset(dataset_path=Path('/tmp'), split='train', train_size=20, image_transform=vision_transform)
dumb_dataset

2023-12-04 18:13:36.240 | DEBUG    | src.datasets.base:__init__:76 - Dataset ImageCaption path: /tmp | Number of samples: 20


In [17]:
dumb_dataset[0][0].shape, dumb_dataset[0][1], caption_transform(dumb_dataset[0][1])

/Users/vladimirzaigrajew/Documents/projects/ClipAligner/.venv/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


(torch.Size([3, 224, 224]),
 'Lorem ipsum ',
 {'input_ids': tensor([[    0,   574, 43375,  1437,  7418,   783,   385, 37194,  2662,   524,
            594,     6,  2764, 41147,   594,   710, 46468,   118,     2,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,   

In [18]:
dataloader = torch.utils.data.DataLoader(dumb_dataset, batch_size=4, shuffle=True)
len(dataloader)

5

In [19]:
for images, captions in dataloader:
    encoded_captions = caption_transform(captions)
    print(images.shape, len(captions), encoded_captions['input_ids'].shape, encoded_captions['attention_mask'].shape)

torch.Size([4, 3, 224, 224]) 4 torch.Size([4, 128]) torch.Size([4, 128])
torch.Size([4, 3, 224, 224]) 4 torch.Size([4, 128]) torch.Size([4, 128])
torch.Size([4, 3, 224, 224]) 4 torch.Size([4, 128]) torch.Size([4, 128])
torch.Size([4, 3, 224, 224]) 4 torch.Size([4, 128]) torch.Size([4, 128])
torch.Size([4, 3, 224, 224]) 4 torch.Size([4, 128]) torch.Size([4, 128])
